In [2]:
import fitz, re
from pix2text import Pix2Text

[WARNING 2025-07-25 00:10:20,561 _showwarnmsg:109] d:\User\ProjectGithub\hiepnguyenn-99\RAG-Solve-Math\ragmathenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 


# giao_trinh_4.pdf

In [3]:
# trích suất trang đầu phần 1.1 Các khái niệm cơ bản

# kiểm tra giao nhau
def intersects(b1, b2):
    x0, y0, x1, y1 = b1
    X0, Y0, X1, Y1 = b2
    return not (x0 > X1 or y0 > Y1 or x1 < X0 or y1 < Y0)

# phát hiện công thức và chuyển thành latex
def extract_text_and_formula(page):
    layout = page.get_text(
        'dict',
        sort=True
        )
    dpi = 150
    pix = page.get_pixmap(dpi=dpi)
    img = pix.pil_image() # chuyển về ảnh pil
    zoom = dpi/72.0 # 1 point = 1/72 inch (pt->px)
    mat_forward = fitz.Matrix(zoom, zoom)
    mat_inverse = ~mat_forward # ma trận nghịch đảo (px->pt)

    # phát hiện công thức trong ảnh
    page_obj = p2t.recognize_page(
        img,
        file_type='text_formula',
        resized_shape=1024,
        return_text=False,
        save_debug_res=None
    )

    formulas = []
    if page_obj:
        formulas = [
            {
                'bbox_pt': fitz.Rect(f.box) * mat_inverse, # chuyển luôn về đơn vị point
                'latex': f.text
            }
            for f in page_obj.elements if f.type.name == "FORMULA"
        ]

    if not formulas:
        return layout
    
    seen_formulas = set()
    for block in layout.get('blocks', []):
        if block['type'] != 0: # chuỗi văn bản có type == 0
            continue
        
        for line in block.get('lines', []):
            line_bbox = fitz.Rect(line['bbox']) # pt
            relevant_formulas = [f for f in formulas if intersects(line_bbox, f['bbox_pt'])]
            if not relevant_formulas:
                continue

            new_spans = []
            original_spans = line['spans']
            # xác định span cần loại vì có ít hơn (cải thiện hiệu suất)
            spans_to_remove_idx = set()
            for i, span in enumerate(original_spans):
                sb = fitz.Rect(span['bbox'])
                for f in relevant_formulas:
                    r_pt = f['bbox_pt']
                    overlap = abs(sb & r_pt)
                    if overlap > 0.8 * abs(sb):
                        spans_to_remove_idx.add(i)
                        break

            for i, span in enumerate(original_spans):
                if i not in spans_to_remove_idx:
                    new_spans.append(span)

            for f in relevant_formulas:
                if f['latex'] not in seen_formulas:
                    new_spans.append({
                        'type': 'formula',
                        'text': f"$$ {f['latex']} $$",
                        'bbox': f['bbox_pt']
                    })
                    seen_formulas.add(f['latex'])
            line['spans'] = sorted(new_spans, key=lambda s: s['bbox'][0])
    return layout

# chuyển pdf về markdown
def render_md(layout):
    md = ''
    for block in layout['blocks']:
        for line in block['lines']:
            spans = line['spans']
            line_text = ''
            for i, s in enumerate(spans):
                text = s['text']
                if i == 0:
                    line_text += text
                else:
                    pre = spans[i-1]['text']
                    if pre and pre[-1] != ' ' and text and text[0] != ' ':
                        line_text += ' ' + text
                    else:
                        line_text += text
            md += line_text + '\n'
        md += '\n'
    return md

providers = ['CUDAExecusionProvider', 'CPUExecutionProvider']

p2t = Pix2Text.from_config(
    analyzer_config={
        "model_name": "mfd-pro",
        "model_backend": "onnx",
        "providers": providers,
    },
    formula_ocr_config={
        "model_name": "mfr-pro",
        "model_backend": "onnx",
        "providers": providers,
    }
)

doc = fitz.open('source/giao_trinh_4.pdf')
page = doc.load_page(11)
layout = extract_text_and_formula(page)
md = render_md(layout)
print(md)

[INFO] 2025-07-25 00:17:41,769 [RapidOCR] base.py:24: Using engine_name: onnxruntime
[INFO] 2025-07-25 00:17:41,774 [RapidOCR] main.py:55: Using C:\Users\admin\AppData\Roaming\cnstd\1.2\ppocr\ch_PP-OCRv5_det\ch_PP-OCRv5_det_infer.onnx
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



0: 1024x736 7 titles, 11 plain texts, 1 abandon, 4 isolate_formulas, 953.1ms
Speed: 0.0ms preprocess, 953.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 736)
Loading C:\Users\admin\AppData\Roaming\pix2text\1.1\mfd-1.5-onnx\pix2text-mfd-1.5.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

WARNING imgsz=[649, 1024] must be multiple of max stride 32, updating to [672, 1024]
0: 672x1024 (no detections), 510.4ms
Speed: 4.5ms preprocess, 510.4ms inference, 0.6ms postprocess per image at shape (1, 3, 672, 1024)


0it [00:00, ?it/s]



WARNING imgsz=[109, 1024] must be multiple of max stride 32, updating to [128, 1024]
0: 128x1024 2 embeddings, 95.9ms
Speed: 0.9ms preprocess, 95.9ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)


100%|██████████| 1/1 [00:02<00:00,  2.68s/it]

...

0: 288x1024 2 embeddings, 215.5ms
Speed: 1.6ms preprocess, 215.5ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 1024)


100%|██████████| 1/1 [00:00<00:00,  4.63it/s]



WARNING imgsz=[1447, 1024] must be multiple of max stride 32, updating to [1472, 1024]
0: 1472x1024 1 embedding, 1290.2ms
Speed: 13.2ms preprocess, 1290.2ms inference, 1.4ms postprocess per image at shape (1, 3, 1472, 1024)


100%|██████████| 1/1 [00:00<00:00,  4.99it/s]


Chương 1

Ma trận

Nội dung

• Định nghĩa và ví dụ.

• Các phép biến đổi sơ cấp.

• Các phép toán đối với ma trận.

• Hạng của ma trận.

• Ma trận nghịch đảo.

1.1
Các khái niệm cơ bản

Định nghĩa 1.1 (Ma trận) .
Ma trận cỡ m × n là một bảng số(thực hoặc phức) hình chữnhật có m hàng và n cột.

$$ A=\left( \begin{matrix} {a_{1 1}} & {\dots} & {a_{1 j}} & {\dots} & {a_{1 n}} \\ {\dots} & {\dots} & {\dots} & {\dots} & {\dots} \\ {a_{i 1}} & {\dots} & {a_{i j}} & {\dots} & {a_{i n}} \\ {\dots} & {\dots} & {\dots} & {\dots} & {\dots} \\ {a_{m 1}} & {\dots} & {a_{m j}} & {\dots} & {a_{m n}} \\ \end{matrix} \right)
 $$











































Ví dụ1.1

$$ A=\left( \begin{matrix} {3} & {4} & {1} \\ {2} & {0} & {5} \\ \end{matrix} \right)_{2 \times3} , B=\left( \begin{matrix} {1+i} & {2} \\ {3-i} & {4 i} \\ \end{matrix} \right)
 $$


















A là ma trận cỡ 2 × 3 có 2 hàng và 3 cột. Các phần tửcủa ma trận A :

$$ a_{1 1}=3 , a_{1 2}=4 , a_{1 3}=1 , a_{2 1}=2 , a_{2 2}

In [6]:
def render_md(layout):
    md = ''
    # mục con của chương
    subchapter = re.compile(r'^\d+\.\d+.*')

    for block in layout['blocks']:
        for line in block['lines']:
            spans = line['spans']
            line_text = ''
            for i, s in enumerate(spans):
                text = s['text']
                if i == 0:
                    line_text += text
                # xử lý dấu cách
                else:
                    pre = spans[i-1]['text']
                    if pre and pre[-1] != ' ' and text and text[0] != ' ':
                        line_text += ' ' + text
                    else:
                        line_text += text

            # xử lý header thêm #
            stripped = line_text.lstrip() # loại bỏ khoảng trắng bên trái
            if stripped.startswith('Chương'):
                line_text = '# ' + stripped
            elif subchapter.match(stripped):
                line_text = '## ' + stripped
            elif stripped.startswith('Định nghĩa'):
                line_text = '#### ' + stripped

            md += line_text + '\n'
        md += '\n'
    return md

md = render_md(layout)
print(md)

# Chương 1

Ma trận

Nội dung

• Định nghĩa và ví dụ.

• Các phép biến đổi sơ cấp.

• Các phép toán đối với ma trận.

• Hạng của ma trận.

• Ma trận nghịch đảo.

## 1.1
Các khái niệm cơ bản

#### Định nghĩa 1.1 (Ma trận) .
Ma trận cỡ m × n là một bảng số(thực hoặc phức) hình chữnhật có m hàng và n cột.

$$ A=\left( \begin{matrix} {a_{1 1}} & {\dots} & {a_{1 j}} & {\dots} & {a_{1 n}} \\ {\dots} & {\dots} & {\dots} & {\dots} & {\dots} \\ {a_{i 1}} & {\dots} & {a_{i j}} & {\dots} & {a_{i n}} \\ {\dots} & {\dots} & {\dots} & {\dots} & {\dots} \\ {a_{m 1}} & {\dots} & {a_{m j}} & {\dots} & {a_{m n}} \\ \end{matrix} \right)
 $$











































Ví dụ1.1

$$ A=\left( \begin{matrix} {3} & {4} & {1} \\ {2} & {0} & {5} \\ \end{matrix} \right)_{2 \times3} , B=\left( \begin{matrix} {1+i} & {2} \\ {3-i} & {4 i} \\ \end{matrix} \right)
 $$


















A là ma trận cỡ 2 × 3 có 2 hàng và 3 cột. Các phần tửcủa ma trận A :

$$ a_{1 1}=3 , a_{1 2}=4 , a_{1 3}=1 , a_{2 1}=2

In [ ]:
with open('extracted/giao_trinh_4.md', 'w', encoding='utf-8') as f:
    for line in md.splitlines():
        if line.strip():
            f.write(line + '\n')

In [ ]:
import fitz, re
from pix2text import Pix2Text

doc = fitz.open('source/giao_trinh_4.pfd')
page = doc.load_page(12)

FileNotFoundError: no such file: 'data/source/giao_trinh_4.pdf'